In [ ]:
# Installing from https://github.com/boudinfl/pke library for Python Keyword extraction
# We use a fixed commit as the later changes might break the code. If it was on pip we would have used exact version number.

# !pip install --quiet git+https://github.com/boudinfl/pke.git@dc4d5f21e0ffe64c4df93c46146d29d1c522476b
# !pip install --quiet flashtext==2.7


!pip install --quiet flashtext==2.7
!pip install git+https://github.com/boudinfl/pke.git
!pip install tokenizers==0.9.4
!pip install sentencepiece==0.1.97
!pip install --no-dependencies transformers==2.9.0
# !pip install transformers==3.4.0
# !pip install transformers==4.1.0
# tokenizers-0.9.4

# !pip install --quiet nltk==3.4.5

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-kjzjt4_v
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-kjzjt4_v
  Resolved https://github.com/boudinfl/pke.git to commit f2d4f5d2252c64d23defccd32fdac8809cfd7ce0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.1 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160676 sha256=ab03bb8c3e39dc4cb6cc937bc507ecc72d6f7eab1012d94520f744d0243b2d71
  Stored in directory: /tmp/pip-ephem-wheel-cache-2bwk2mhv/wheels/d5/46/97/85535b5b449f70b6a3c8d1138ce8587345876891e25bfe7954
Successfully built pke
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/

In [ ]:
# connect your personal google drive to store the trained model
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import textwrap
# Story source - https://byjus.com/kids-learning/moral-stories-the-lion-and-the-mouse/

text = """ Once upon a time, there lived a lion in the dense Amazon rainforest. While he was sleeping by resting his big head on his paws, a tiny little mouse unexpectedly crossed by and ran across the lion’s nose in haste. This woke up the lion and he laid his huge paw angrily on the tiny mouse to kill her.

The poor mouse begged the lion to spare her this time and she would pay him back on some other day. Hearing this, the lion was amused and wondered how could such a tiny creature ever help him. But he was in a good mood and in his generosity he finally let the mouse go.

A few days later, a hunter set a trap for the lion while the big animal was stalking for prey in the forest. Caught in the toils of a hunter’s net, the lion found it difficult to free himself and roared loudly in anger.

As the mouse was passing by, she heard the roar and found the lion struggling hard to free himself from the hunter’s net. The little creature quickly ran towards the lion’s trap that bound him and she gnawed the net with her sharp teeth until the net tore apart. Slowly she made a big hole in the net and soon the lion was able to free himself from the hunter’s trap.

The lion thanked the little mouse for her help and the mouse reminded him that she had finally repaid the lion for sparing her life before. Thereafter, the lion and the mouse became good friends and lived happily in the forest. """

wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list:
  print(element)

 Once upon a time, there lived a lion in the dense Amazon rainforest. While he was sleeping by resting his big head on his paws, a tiny little mouse
unexpectedly crossed by and ran across the lion’s nose in haste. This woke up the lion and he laid his huge paw angrily on the tiny mouse to kill her.
The poor mouse begged the lion to spare her this time and she would pay him back on some other day. Hearing this, the lion was amused and wondered how
could such a tiny creature ever help him. But he was in a good mood and in his generosity he finally let the mouse go.  A few days later, a hunter set
a trap for the lion while the big animal was stalking for prey in the forest. Caught in the toils of a hunter’s net, the lion found it difficult to
free himself and roared loudly in anger.  As the mouse was passing by, she heard the roar and found the lion struggling hard to free himself from the
hunter’s net. The little creature quickly ran towards the lion’s trap that bound him and she gnawed 

In [ ]:
def get_keywords(text):
    out=[]
    try:
        # extractor = pke.unsupervised.MultipartiteRank()
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text,language='en')
        grammar = r"""
                NP:
                    {<NOUN|PROPN>+}
            """
        extractor.ngram_selection(n=1)
        extractor.grammar_selection(grammar=grammar)
        # pos = {'VERB', 'ADJ', 'NOUN'}
        # pos ={'NOUN'}
        # stoplist = list(string.punctuation)
        # stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        # stoplist += stopwords.words('english')
        # extractor.candidate_selection(n=1,pos=pos, stoplist=stoplist)
        extractor.candidate_selection(n=1)

        extractor.candidate_weighting(window=3,
                                      use_stems=False)

        keyphrases = extractor.get_n_best(n=30)


        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

keywords = get_keywords(text)[:8]
print ("keywords: ",keywords)

keywords:  ['lion', 'mouse', 'amazon', 'net', 'hunter', 'tiny', 'free', 'big']


In [ ]:
sentences = tokenize_sentences(text)
print (sentences)

['Once upon a time, there lived a lion in the dense Amazon rainforest.', 'While he was sleeping by resting his big head on his paws, a tiny little mouse unexpectedly crossed by and ran across the lion’s nose in haste.', 'This woke up the lion and he laid his huge paw angrily on the tiny mouse to kill her.', 'The poor mouse begged the lion to spare her this time and she would pay him back on some other day.', 'Hearing this, the lion was amused and wondered how could such a tiny creature ever help him.', 'But he was in a good mood and in his generosity he finally let the mouse go.', 'A few days later, a hunter set a trap for the lion while the big animal was stalking for prey in the forest.', 'Caught in the toils of a hunter’s net, the lion found it difficult to free himself and roared loudly in anger.', 'As the mouse was passing by, she heard the roar and found the lion struggling hard to free himself from the hunter’s net.', 'The little creature quickly ran towards the lion’s trap that

In [ ]:
from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=False)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
pprint (keyword_sentence_mapping)

{'amazon': ['Once upon a time, there lived a lion in the dense Amazon '
            'rainforest.'],
 'big': ['Slowly she made a big hole in the net and soon the lion was able to '
         'free himself from the hunter’s trap.',
         'A few days later, a hunter set a trap for the lion while the big '
         'animal was stalking for prey in the forest.',
         'While he was sleeping by resting his big head on his paws, a tiny '
         'little mouse unexpectedly crossed by and ran across the lion’s nose '
         'in haste.'],
 'free': ['Slowly she made a big hole in the net and soon the lion was able to '
          'free himself from the hunter’s trap.',
          'Caught in the toils of a hunter’s net, the lion found it difficult '
          'to free himself and roared loudly in anger.',
          'As the mouse was passing by, she heard the roar and found the lion '
          'struggling hard to free himself from the hunter’s net.'],
 'hunter': ['Slowly she made a big hole 

In [ ]:
import os
import zipfile

bert_wsd_pytorch = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6.zip"
extract_directory = "/content/gdrive/My Drive"

extracted_folder = bert_wsd_pytorch.replace(".zip","")

#  If unzipped folder exists don't unzip again.
if not os.path.isdir(extracted_folder):
  with zipfile.ZipFile(bert_wsd_pytorch, 'r') as zip_ref:
      zip_ref.extractall(extract_directory)
else:
  print (extracted_folder," is extracted already")

FileNotFoundError: ignored